### ECE-GY 6123
### Nikita Makarov, Fall 2024

### Homework 3

Problem 1a:

In [2]:
import numpy as np
import cv2

# Create Gaussian Pyramid function
def gaussian_pyramid(im_data: np.ndarray, J: int) -> list[np.ndarray]:
    '''
    Compute and return the J-level Gaussian pyramid decomposition of a grayscale image.

    Parameters:
    im_data: 2D numpy array of the image data in grayscale
    J: int of the Gaussian pyramid level

    Returns:
    G: list of size J filled with 2D numpy arrays of the Gaussian pyramid, with the first element being level J
    '''

    # Create Gaussian pyramid structure
    G = [im_data] # Create list with first element as level J

    # Iterate over all remaining levels
    for i in range(1,J):
        width = int(im_data.shape[1]/2) # Calculate new width
        height = int(im_data.shape[0]/2) # Calculate new height
        im_data = cv2.resize(im_data,(width,height),interpolation=cv2.INTER_LINEAR) # Get next layer of image data
        G.append(im_data) # Append new layer to end of list

    return G

# Create Laplacian Pyramid function
def laplacian_pyramid(im_data: np.ndarray, J: int) -> list[np.ndarray]:
    '''
    Compute and return the J-level Laplacian pyramid decomposition of a grayscale image.

    Parameters:
    im_data: 2D numpy array of the image data in grayscale
    J: int of the Laplacian pyramid level

    Returns:
    L: list of size J filled with 2D numpy arrays of the Laplacian pyramid, with the first element being level J
    '''


image = cv2.imread('coins.png', cv2.IMREAD_GRAYSCALE)
gaussian_pyramid(image, 3)
pass